In [1]:
import meshplot as mp
import numpy as np
from skimage.measure import marching_cubes
import networkx as nx

In [2]:
centerline = nx.Graph()
centerline.add_node(0, position=np.array([0,0,0]))
centerline.add_node(1, position=np.array([0,0.5,0]))
centerline.add_node(2, position=np.array([0,1,0.2]))

centerline.add_edges_from([(0,1), (1,2)])

In [3]:
p = mp.plot( np.array([ data['position'] for node, data in centerline.nodes(data=True)]) , shading={'point_color': 'red', 'point_size':0.2},return_plot=True)

for arista in centerline.edges:
    p.add_lines( centerline.nodes[arista[0]]['position'], centerline.nodes[arista[1]]['position'])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.5,…

La sdf del cilindro, normalizado entre 0 y 1

In [5]:
def sdfSphere( p ):
    return np.linalg.norm(p) - 1


def voxels( a, b, samples=30):
    grid = []

    for x in np.linspace(-1, 1, samples):
        plano = []
        for y in np.linspace(-1, 1, samples):
            row = []
            for z in np.linspace(-1, 1, samples):
                row.append( sdfSphere(np.array([x,y,z])) )
            plano.append(row)
        grid.append(plano)

    grid = np.array(grid)
    return grid
    
def mesh( voxels ):
    verts, faces, normals, values = marching_cubes( voxels, 0)
    return verts, faces

In [6]:
verts, faces = mesh( voxels(np.array( [0.4, 0 , 0] ), np.array( [0,0,0])))
mp.plot( verts, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(14.499999…

In [ ]:
def sdfUnion( centerline, p ):
    .basisIn( [] )

float sdCylinder(vec3 p, vec3 a, vec3 b, float r)
{
    vec3  ba = b - a;
    vec3  pa = p - a;
    float baba = dot(ba,ba);
    float paba = dot(pa,ba);
    float x = length(pa*baba-ba*paba) - r*baba;
    float y = abs(paba-baba*0.5)-baba*0.5;
    float x2 = x*x;
    float y2 = y*y*baba;
    
    float d = (max(x,y)<0.0)?-min(x2,y2):(((x>0.0)?x2:0.0)+((y>0.0)?y2:0.0));
    
    return sign(d)*sqrt(abs(d))/baba;
}

float sdCappedCylinder2( vec3 p, vec2 h )
{   
  vec2 d = abs(vec2(length(p.xz),p.y)) - h;
  return min(max(d.x,d.y),0.0) + length(max(d,0.0));
}

float map( in vec3 pos )
{
    //return sdCylinder(pos, vec3(0.4,0.0,0.0), vec3(-0.4,0,0), 0.3 );
    mat3 rot;
    float angle = 1.56;
    rot[0].xyz = vec3(cos(angle), sin(angle), 0);
    rot[1].xyz = vec3(-sin(angle), cos(angle), 0);
    rot[2].xyz = vec3(0, 0, 1);
    return sdCappedCylinder2( rot * pos, vec2(0.1,0.1));
}

// https://iquilezles.org/articles/normalsSDF
vec3 calcNormal( in vec3 pos )
{
    vec2 e = vec2(1.0,-1.0)*0.5773;
    const float eps = 0.0005;
    return normalize( e.xyy*map( pos + e.xyy*eps ) + 
					  e.yyx*map( pos + e.yyx*eps ) + 
					  e.yxy*map( pos + e.yxy*eps ) + 
					  e.xxx*map( pos + e.xxx*eps ) );
}
    
#if HW_PERFORMANCE==0
#define AA 1
#else
#define AA 3
#endif

void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
     // camera movement	
	float an = iTime * 0.3;
	vec3 ro = vec3( 1.0*cos(an), 0.4, 1.0*sin(an) );
    vec3 ta = vec3( 0.0, 0.0, 0.0 );
    // camera matrix
    vec3 ww = normalize( ta - ro );
    vec3 uu = normalize( cross(ww,vec3(0.0,1.0,0.0) ) );
    vec3 vv = normalize( cross(uu,ww));

        
    vec3 tot = vec3(0.0);
    
    #if AA>1
    for( int m=0; m<AA; m++ )
    for( int n=0; n<AA; n++ )
    {
        // pixel coordinates
        vec2 o = vec2(float(m),float(n)) / float(AA) - 0.5;
        vec2 p = (-iResolution.xy + 2.0*(fragCoord+o))/iResolution.y;
        #else    
        vec2 p = (-iResolution.xy + 2.0*fragCoord)/iResolution.y;
        #endif

	    // create view ray
        vec3 rd = normalize( p.x*uu + p.y*vv + 1.5*ww );

        // raymarch
        const float tmax = 3.0;
        float t = 0.0;
        for( int i=0; i<256; i++ )
        {
            vec3 pos = ro + t*rd;
            float h = map(pos);
            if( h<0.0001 || t>tmax ) break;
            t += h;
        }
        
    
        // shading/lighting	
        vec3 col = vec3(0.0);
        if( t<tmax )
        {
            vec3 pos = ro + t*rd;
            vec3 nor = calcNormal(pos);
            float dif = clamp( dot(nor,vec3(0.57703)), 0.0, 1.0 );
            float amb = 0.5 + 0.5*dot(nor,vec3(0.0,1.0,0.0));
            col = vec3(0.2,0.3,0.4)*amb + vec3(0.8,0.7,0.5)*dif;
        }

        // gamma        
        col = sqrt( col );
	    tot += col;
    #if AA>1
    }
    tot /= float(AA*AA);
    #endif

	fragColor = vec4( tot, 1.0 );
}